# Cleaning

In [2]:
!pip install contractions
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=19c429560a04f967a4e8d940b5c24fa28c2d5fbc4d787145c205005e910ec9a7
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
import os
import pandas as pd
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from langdetect import detect

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
run_mode = 'kaggle'
if run_mode == 'kaggle':
    train = pd.read_csv('/kaggle/input/fake-news/train.csv')
    test = pd.read_csv('/kaggle/input/fake-news/test.csv')
else: # local
    train = pd.read_csv('../../Dataset_Original/UTK/train.csv')
    test = pd.read_csv('../../Dataset_Original/UTK/test.csv')
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Remove the author column

In [8]:
train.drop(columns=['author'], inplace=True)
train.head()

,id,title,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
train.head()

,id,title,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


Null values

In [10]:
print(train['label'].value_counts())

label
1    10413
0    10387
Name: count, dtype: int64


In [11]:
train.isnull().sum()

id         0
title    558
text      39
label      0
dtype: int64

In [12]:
train = train.dropna(subset=['text'])

Outliers

In [13]:
train[train['text'].str.len() <= 2].head()

,id,title,text,label
82,82,Huma’s Weiner Dogs Hillary,,1
169,169,Mohamad Khweis: Another “Virginia Man” (Palest...,,1
295,295,A Connecticut Reader Reports Record Voter Regi...,,1
470,470,BULLETIN: There ARE Righteous Jews For Trump!;...,,1
592,592,Is your promising internet career over now Vin...,,1


In [14]:
train = train[train['text'].str.len() > 2]

In [15]:
train[train['text'] == "source Add To The Conversation Using Facebook Comments"]['text']

519      source Add To The Conversation Using Facebook ...
3206     source Add To The Conversation Using Facebook ...
4726     source Add To The Conversation Using Facebook ...
4781     source Add To The Conversation Using Facebook ...
5052     source Add To The Conversation Using Facebook ...
5539     source Add To The Conversation Using Facebook ...
6891     source Add To The Conversation Using Facebook ...
7055     source Add To The Conversation Using Facebook ...
7628     source Add To The Conversation Using Facebook ...
7946     source Add To The Conversation Using Facebook ...
8386     source Add To The Conversation Using Facebook ...
8594     source Add To The Conversation Using Facebook ...
10095    source Add To The Conversation Using Facebook ...
10343    source Add To The Conversation Using Facebook ...
11975    source Add To The Conversation Using Facebook ...
12212    source Add To The Conversation Using Facebook ...
12615    source Add To The Conversation Using Facebook .

In [16]:
train = train[train['text'] != "source Add To The Conversation Using Facebook Comments"]

# Preprocessing

In [17]:
train['combined'] = train['title'] + ' ' + train['text']

In [18]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])
def tokenize_text(text):
    return word_tokenize(text)

In [19]:
def preprocess(df):
    # Convert 'combined' column to strings
    df['combined'] = df['combined'].astype(str)
    
    # Lowercase
    df['combined'] = df['combined'].str.lower()

    # Expanding Contractions
    df['combined'] = df['combined'].apply(lambda x: contractions.fix(x))

    # Removing the URLs
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))

    # Removing the HTML tags
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'<.*?>', '', x))

    # Removing the numbers
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'\d+', '', x))

    # Removing the extra whitespaces
    df['combined'] = df['combined'].apply(lambda x: x.strip())

    # Removing the punctuation
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Removing the stopwords
    df['combined'] = df['combined'].apply(remove_stopwords)

    # Tokenization
    df['combined'] = df['combined'].apply(tokenize_text)

    return df

In [20]:
train = preprocess(train)

In [21]:
train['combined'].dropna(inplace=True)

# Model

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Import model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold

In [23]:
X_train = train['combined']
y_train = train['label']

In [24]:
X_train.head()

0    [house, dem, aide, even, see, comeys, letter, ...
1    [flynn, hillary, clinton, big, woman, campus, ...
2    [truth, might, get, fired, truth, might, get, ...
3    [civilians, killed, single, us, airstrike, ide...
4    [iranian, woman, jailed, fictional, unpublishe...
Name: combined, dtype: object

In [25]:
vectorizer = TfidfVectorizer()
X_train = X_train.apply(' '.join)
X_train = vectorizer.fit_transform(X_train)

In [26]:
# Split train/validation data
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

## Param tuning

# 1

In [27]:
# Best model
clf =RandomForestClassifier(
                        n_estimators=200,
                        max_depth=100,
                        min_samples_split=20,
                        max_features= 'sqrt' ,
                        
                        random_state=42
)

clf.fit(X_train_split, y_train_split)

RandomForestClassifier(max_depth=100, min_samples_split=20, n_estimators=200,
                       random_state=42)

In [28]:
# Training performance
y_train_pred = clf.predict(X_train_split)
print(classification_report(y_train_split, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8309
           1       1.00      1.00      1.00      8215

    accuracy                           1.00     16524
   macro avg       1.00      1.00      1.00     16524
weighted avg       1.00      1.00      1.00     16524



In [29]:
# Validate performance
y_val_pred = clf.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2078
           1       0.96      0.92      0.94      2054

    accuracy                           0.94      4132
   macro avg       0.94      0.94      0.94      4132
weighted avg       0.94      0.94      0.94      4132



# 2

In [39]:
clf =RandomForestClassifier(
                        n_estimators=1000,
                        max_depth=200,
                        min_samples_split=50,
                        max_features= 'sqrt' ,
                        
                        random_state=42
)
clf.fit(X_train_split, y_train_split)

RandomForestClassifier(max_depth=200, min_samples_split=50, n_estimators=1000,
                       random_state=42)

In [40]:
# Training performance
y_train_pred = clf.predict(X_train_split)
print(classification_report(y_train_split, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8309
           1       1.00      1.00      1.00      8215

    accuracy                           1.00     16524
   macro avg       1.00      1.00      1.00     16524
weighted avg       1.00      1.00      1.00     16524



In [41]:
# Validate performance
y_val_pred = clf.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      2078
           1       0.96      0.92      0.94      2054

    accuracy                           0.94      4132
   macro avg       0.94      0.94      0.94      4132
weighted avg       0.94      0.94      0.94      4132



In [30]:
## Train on full dataset

In [42]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=200, min_samples_split=50, n_estimators=1000,
                       random_state=42)

# Submission

In [43]:
test['combined'] = test['title'] + ' ' + test['text']
test = preprocess(test)
X_test = test['combined']
X_test = X_test.apply(' '.join)
X_test = vectorizer.transform(X_test)
id_test = test['id']
y_test_pred = clf.predict(X_test)

In [44]:
# Create a DataFrame
submission_df = pd.DataFrame({'id': id_test, 'label': y_test_pred})
# Save the DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/rf_submission.csv', index=False)

In [45]:
from joblib import dump
dump(clf, 'svm_model.joblib')

['svm_model.joblib']